## Census API - ACS Example

Using the American Community Survey to examine some demographic and economic trends at the U.S. county level.

List of variables from the 5-year ACS are found here: https://api.census.gov/data/2015/acs5/variables.html

The vincent example requires two topo.json files: States and Counties

In [1]:
import requests
import pandas as pd

import config
key = config.census_key

# For mapping results
import vincent
vincent.core.initialize_notebook()

In [2]:
base = 'http://api.census.gov/data/'
years = ['2015']#['2009', '2012', '2015']
variables = {'NAME':'Name',
             'B01001_001E': 'Population total',
             'B19013_001E': 'Real Median Income',}
v = ','.join(variables.keys())
s = '*'

In [3]:
df = pd.DataFrame()
for y in years:
    url = '{}{}/acs5?get={}&for=county:*&in=state:{}&key={}'.format(base, y, v, s, key)
    r = requests.get(url).json()
    dft = pd.DataFrame(r[1:], columns=r[0])
    dft['Year'] = y
    df = df.append(dft)
df = df.rename(columns=variables).set_index(['Name', 'Year']).sort_index(level='Name')
df.head()

,,Population total,Real Median Income,state,county
Name,Year,,,,
"Abbeville County, South Carolina",2015,24997,34799,45,001
"Acadia Parish, Louisiana",2015,62163,37649,22,001
"Accomack County, Virginia",2015,33115,39412,51,001
"Ada County, Idaho",2015,417501,56356,16,001
"Adair County, Iowa",2015,7426,46526,19,001


### Map the results

Note: to make the example below work, you will first need to save [this](https://raw.githubusercontent.com/wrobstory/vincent_map_data/master/us_counties.topo.json) topo.json file in the same directory as the jupyter notebook.

In [4]:
df['Real Median Income'] = df['Real Median Income'].astype(float)

df['FIPS'] = df['state'] + df['county']
df['FIPS'] = df['FIPS'].astype(int)
df['FIPS'] = df['FIPS'].map(lambda i: str(i).zfill(5))
# County FIP Codes that have changed:
df['FIPS'] = df['FIPS'].str.replace('46102', '46113')

In [5]:
geo_data = [{'name': 'counties',
             'url': 'geo/us_counties.topo.json',
             'feature': 'us_counties.geo'},            
            {'name': 'states',
             'url': 'geo/us_states.topo.json',
             'feature': 'us_states.geo'}
             ]

vis = vincent.Map(data=df, geo_data=geo_data, scale=1100,
                  projection='albersUsa', data_bind='Real Median Income',
                  data_key='FIPS', map_key={'counties': 'properties.FIPS'})

del vis.marks[1].properties.update
vis.marks[0].properties.enter.stroke.value = '#fff'
vis.marks[1].properties.enter.stroke.value = '#000000'
vis.scales['color'].domain = [0, 75000] # Adjust
vis.legend(title='Real Median Income')
vis.to_json('vega.json')

vis.display()